# Langchain

In [60]:
import requests
from langchain_community.chat_models import ChatOllama
from langchain.agents import initialize_agent, Tool, AgentType
from datetime import datetime

In [61]:
# Simulation d'un appelle à une API météo
def get_weather(city):
    temp = 25
    wind = 30
    return f"Il fait {temp}°C avec un vent à {wind} km/h."

def get_time():
    return datetime.now().strftime("%H:%M:%S")

In [62]:
# On crée le Tool LangChain
weather_tool = Tool.from_function(
    func=get_weather,
    name="get_weather",
    description="Donne la météo actuelle pour une ville spécifiée."
)

# On instancie ton modèle Ollama
#llm = ChatOllama(model="gemma3n:e2b")
#llm = ChatOllama(model="qwen2.5:3b")
llm = ChatOllama(model="phi4-mini:latest")

# On initialise l'agent
agent = initialize_agent(
    tools=[weather_tool],
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    handle_parsing_errors=True,
    agent_kwargs={"prefix": """Si tu peux répondre directement à la question (sans appeler un outil), fais-le. 
N’utilise un outil que si c’est strictement nécessaire. 
Respecte scrupuleusement ce format pour les outils :

Thought: Raisonne brièvement
Action: nom_de_l_outil
Action Input: valeur"""}
)

# On pose une question à l'agent
result = agent.run("Peux-tu me dire la météo actuelle à Paris ?")
print(result)



> Entering new AgentExecutor chain...
Thought: To provide weather information, I'll use a service that delivers current meteorological data for specified locations.
Action: get_weather(city)
Action Input: city=Paris
Observation: get_weather(city) is not a valid tool, try one of [get_weather].
Thought:Question: Peux-tu me dire la météo actuelle à Paris ?
Thought: I need to call the function that will provide current weather information for Paris.
Action: get_weather(city)
Action Input: city=Paris
Observation: get_weather(city) is not a valid tool, try one of [get_weather].
Thought:I apologize; it seems there was an error with my response. Let me correct this and proceed correctly.

Question: Peux-tu me dire la météo actuelle à Paris ?
Thought: I need to call the function that will provide current weather information for Paris.
Action get_weather(city)
Action Input: city=Paris
Observation: Invalid Format: Missing 'Action:' after 'Thought:
Thought:I apologize, my previous message contai

In [63]:
result = agent.run("Combien font 2+2 ?")
print(result)



> Entering new AgentExecutor chain...
Thought: This is a simple arithmetic problem that does not require any tools or external sources.

Action: None required

Observation: Invalid Format: Missing 'Action Input:' after 'Action:'
Thought:Question: How much do 2 + 2 make?

Thought: I already know the answer to this basic math question without needing an action input. The calculation is straightforward and common knowledge, so no tool or further steps are necessary.

Final Answer: Two plus two equals four (4).

> Finished chain.
Two plus two equals four (4).


Analyse :
Beaucoup de problémes soit il utilise trop l'outils soit il y a des erreurs de parsing. Pas très stable. Ou alors la réponse est en anglais.

# A la main

In [64]:
import ollama
import re
import json

In [65]:
def get_pokemon_info(pokemon_name):
    response = requests.get(f"https://pokeapi.co/api/v2/pokemon/{pokemon_name.lower()}")
    if response.status_code == 200:
        data = response.json()
        return f"{data['name'].title()} - Poids: {data['weight']/10}kg, Taille: {data['height']/10}m"
    return f"Pokémon '{pokemon_name}' non trouvé"

In [66]:
available_tools = {
    "get_weather": {
        "description": "Donne la météo actuelle pour une ville spécifiée.",
        "format": """{"name": "get_weather", "params": ["ville"]}""",
        "tool_function": get_weather
    },
    "get_time": {
        "description": "Donne l'heure actuelle.",
        "format": """{"name": "get_time", "params": []}""",
        "tool_function": get_time
    },
    "get_pokemon_info": {
        "description": "Récupère des informations sur un Pokémon.",
        "format": """{"name": "get_pokemon_info", "params": ["nom_pokemon"]}""",
        "tool_function": get_pokemon_info
    }
}

In [67]:
def ollama_llm_tool(question, available_tools):
    tools_description = ""
    for tool_name, tool_info in available_tools.items():
        tools_description += f"- Nom: {tool_name}\n"
        tools_description += f"  Description: {tool_info['description']}\n"
        tools_description += f"  Format: {tool_info['format']}\n\n"
    
    formatted_prompt = f"""Tu es un assistant intelligent qui doit analyser les questions des utilisateurs et déterminer si elles nécessitent l'utilisation d'un outil spécifique.

OUTILS DISPONIBLES:
{tools_description}

QUESTION DE L'UTILISATEUR:
{question}

    INSTRUCTIONS:
    1. Analyse la question de l'utilisateur
    2. Détermine si elle nécessite l'utilisation d'un des outils disponibles
    3. Réponds UNIQUEMENT par:
    - Format JSON: {{"tools": [{{"name": "nom_outil1", "params": ["paramètre1", "paramètre2",...]}},{{"name": "nom_outil2", "params": ["paramètre1",...]}}]}} si des outils sont nécessaires
    - Format JSON: {{"tools": "NONE"}} si tu peux répondre directement sans outil

    EXEMPLES:
    - Question: "Quel temps fait-il à Paris ?" → Réponse: {{"tools": [{{"name": "get_weather", "params": ["Paris"]}}]}}
    - Question: "Quelle heure est-il ?" → Réponse: {{"tools": [{{"name": "get_time", "params": []}}]}}
    - Question: "Dis-moi l'heure et la météo à Lyon" → Réponse: {{"tools": [{{"name": "get_time", "params": []}}, {{"name": "get_weather", "params": ["Lyon"]}}]}}
    - Question: "Combien font 2+2 ?" → Réponse: {{"tools": "NONE"}}
    - Question: "Quelle est la taille de Pikachu ?" → Réponse: {{"tools": [{{"name": "get_pokemon_info", "params": ["Pikachu"]}}]}}


    RÉPONSE:"""
    
    response = ollama.chat(
        model="phi4-mini:latest",
        messages=[{'role': 'user', 'content': formatted_prompt}]
    )

    response_content = response['message']['content']
    final_answer = re.sub(r'<think>.*?</think>',
                          '',
                          response_content,
                          flags=re.DOTALL).strip()
    
    return final_answer

In [68]:
r1 = ollama_llm_tool("Peux-tu me dire la météo actuelle à Tallard et l'heur actuelle ?", available_tools)
r1

'{"tools": [{"name": "get_weather", "params": ["Tallard"]}, {"name": "get_time", "params": []}]}'

In [69]:
r2 = ollama_llm_tool("Tu est plutot chien ou chat ?", available_tools)
r2

'{"tools": "NONE"}'

In [70]:
def parse_tool_response(response):
    first_brace = response.find('{')
    last_brace = response.rfind('}')

    # Si on trouve les deux accolades
    if first_brace != -1 and last_brace != -1 and first_brace < last_brace:
        json_str = response[first_brace:last_brace + 1]
        print(f"JSON extrait: {json_str}")
        
        try:
            # Tenter de parser le JSON
            parsed = json.loads(json_str)
            print(f"JSON parsé avec succès: {parsed}")
        
        # Si il y a une erreur pour ne pas couper le programme on return {'tools': 'NONE'}
        except json.JSONDecodeError as e:
            print(f"Erreur de parsing JSON: {e}")
            return {'tools': 'NONE'}

    return parsed


In [71]:
tools_to_call = parse_tool_response(r1)
tools_to_call

JSON extrait: {"tools": [{"name": "get_weather", "params": ["Tallard"]}, {"name": "get_time", "params": []}]}
JSON parsé avec succès: {'tools': [{'name': 'get_weather', 'params': ['Tallard']}, {'name': 'get_time', 'params': []}]}


{'tools': [{'name': 'get_weather', 'params': ['Tallard']},
  {'name': 'get_time', 'params': []}]}

In [72]:
def execute_tools(tools_to_call, available_tools):
    if tools_to_call.get("tools") == "NONE":
        return None

    results = []
    for tool in tools_to_call.get("tools", []):
        tool_name = tool["name"]
        params = tool["params"]
        
        if tool_name in available_tools:
            tool_info = available_tools[tool_name]
            tool_function = tool_info["tool_function"]
            tool_description = tool_info["description"]
            
            result = tool_function(*params)
            results.append({
                "tool_name": tool_name,
                "description": tool_description,
                "params": params,
                "result": result
            })
        else:
            print(f"Outil {tool_name} non reconnu.")
            results.append({
                "tool_name": tool_name,
                "description": "Outil non trouvé",
                "params": params,
                "result": f"Erreur: Outil '{tool_name}' non disponible"
            })
            
    print(f"Résultats des outils exécutés: {results}")
    return results

In [73]:
execute_tools(tools_to_call, available_tools)

Résultats des outils exécutés: [{'tool_name': 'get_weather', 'description': 'Donne la météo actuelle pour une ville spécifiée.', 'params': ['Tallard'], 'result': 'Il fait 25°C avec un vent à 30\u202fkm/h.'}, {'tool_name': 'get_time', 'description': "Donne l'heure actuelle.", 'params': [], 'result': '12:28:57'}]


[{'tool_name': 'get_weather',
  'description': 'Donne la météo actuelle pour une ville spécifiée.',
  'params': ['Tallard'],
  'result': 'Il fait 25°C avec un vent à 30\u202fkm/h.'},
 {'tool_name': 'get_time',
  'description': "Donne l'heure actuelle.",
  'params': [],
  'result': '12:28:57'}]

In [74]:
def run_llm_agent(question, available_tools):

    tools_to_call = parse_tool_response(ollama_llm_tool(question, available_tools))
    tools_results = execute_tools(tools_to_call, available_tools)

    if tools_results is None:
        formatted_prompt = f"""Tu es un assistant intelligent. Réponds à cette question :

QUESTION: {question}

Réponds de manière naturelle et conversationnelle en français."""
    
    else:
        # Formatter les résultats des outils
        tools_summary = ""
        for tool_result in tools_results:
            tool_name = tool_result["tool_name"]
            description = tool_result["description"]
            params = tool_result["params"]
            result = tool_result["result"]
            
            tools_summary += f"- Outil: {tool_name} ({description})\n"
            tools_summary += f"  Paramètres: {params}\n"
            tools_summary += f"  Résultat: {result}\n\n"
        
        formatted_prompt = f"""Tu es un assistant intelligent. L'utilisateur a posé une question et j'ai utilisé des outils pour obtenir des informations.

QUESTION DE L'UTILISATEUR:
{question}

RÉSULTATS DES OUTILS:
{tools_summary}

INSTRUCTIONS:
- Utilise les résultats des outils pour répondre à la question de l'utilisateur
- Réponds de manière naturelle et conversationnelle en français
- Synthétise les informations de manière claire et utile
- Ne mentionne pas les noms techniques des outils, parle naturellement

RÉPONSE:"""
    response = ollama.chat(
        model="phi4-mini:latest",
        messages=[{'role': 'user', 'content': formatted_prompt}]
    )

    response_content = response['message']['content']
    final_answer = re.sub(r'<think>.*?</think>',
                          '',
                          response_content,
                          flags=re.DOTALL).strip()
    
    return final_answer

In [76]:
run_llm_agent("Quelle est la taille de pikatchu ?", available_tools)
# Attention l'api de pokemon ne comprend que l'anglais
run_llm_agent("Quelle température fait il a Paris ?", available_tools)

JSON extrait: {"tools": [{"name": "get_pokemon_info", "params": ["Pikachu"]}]}
JSON parsé avec succès: {'tools': [{'name': 'get_pokemon_info', 'params': ['Pikachu']}]}
Résultats des outils exécutés: [{'tool_name': 'get_pokemon_info', 'description': 'Récupère des informations sur un Pokémon.', 'params': ['Pikachu'], 'result': 'Pikachu - Poids: 6.0kg, Taille: 0.4m'}]
JSON extrait: {"tools": [{"name": "get_weather", "params": ["Paris"]}]}
JSON parsé avec succès: {'tools': [{'name': 'get_weather', 'params': ['Paris']}]}
Résultats des outils exécutés: [{'tool_name': 'get_weather', 'description': 'Donne la météo actuelle pour une ville spécifiée.', 'params': ['Paris'], 'result': 'Il fait 25°C avec un vent à 30\u202fkm/h.'}]


"Il fait actuellement 25°C à Paris avec un vent qui souffle à environ 30 kilomètres par heure. Un temps assez chaud aujourd'hui ! Avez-vous besoin d'autres prévisions ou informations sur la météo ?"

Résultat : Bien meilleur ! Même s'il n'y a pas de callback à répétition comme avec LangChain, le modèle tourne moins en boucle et se concentre davantage sur ses tâches. Plus précis, en tout cas avec un petit modèle. En amélioration : le raisonnment pourrait etre fait en anglais pour éviter les erreurs de traduction puis la réponse traduite en français avec un llm
